<a target="_blank" href="https://colab.research.google.com/github/bettercodepaul/nixtla_intro_workshop/blob/main/Introduction_to_NeuralForecast_TimeGPT.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Forecasting with Nixtla's NeuralForecast and TimeGPT

This notebook walks you through the very basics of forecasting time series with Nixtla's NeuralForecast and its foundation model TimeGPT.

## Install and import necessary libraries

We use [Polars](https://docs.pola.rs/) for data wrangling, [Plotly](https://plotly.com/python/plotly-express/) for visualizations and Nixtla's [NeuralForecast](https://nixtlaverse.nixtla.io/neuralforecast/docs/getting-started/introduction.html) and [TimeGPT](https://www.nixtla.io/docs/introduction/introduction) for time series forecasting.

In [ ]:
pip -q install neuralforecast nixtla polars plotly

In [ ]:
import polars as pl
import plotly.express as px
from datetime import date

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
from nixtla import NixtlaClient

from utilsforecast.plotting import plot_series

## Create a TimeGPT account and generate your API key

- Visit [dashboard.nixtla.io](https://dashboard.nixtla.io) to activate your free trial and create an account.
- Sign in using Google, GitHub, or your email.
- Navigate to API Keys in the menu and select Create New API Key.
- Your new API key will appear on the screen. Copy this key using the button on the right.
- Add it to your Google Colab secrets on the left

In [ ]:
# Get Nixtla API from user secrets on Google Colab
# If you are not using Google Colab, replace this with your own method
from google.colab import userdata
nixtla_api_key = userdata.get('NIXTLA_API_KEY')

In [ ]:
nixtla_client = NixtlaClient(
    api_key=nixtla_api_key
)
assert nixtla_client.validate_api_key()

## Load the data

The data for this walk through is simple monthly sales data from various countries.

In [ ]:
Y_df = pl.read_parquet("https://github.com/bettercodepaul/nixtla_intro_workshop/raw/refs/heads/main/retail_sales.parquet")
Y_df.sample(5)

In [ ]:
nixtla_client.plot(Y_df)

## Forecast using NeuralForecast

In [ ]:
# Split the data into training and test sets
last_training_year = 2017
validation_years = 2
Y_train_df = Y_df.filter(pl.col("ds").dt.year().le(last_training_year))
Y_validation_df = Y_df.filter(pl.col("ds").dt.year().is_between(last_training_year + 1, last_training_year + validation_years))

In [ ]:
## Make a forecast with the neural network models NHITS and NBEATS
horizon = Y_validation_df.select(pl.col("ds").n_unique()).item(0, 0)
models = [NBEATS(input_size=2 * horizon, h=horizon, max_steps=100, enable_progress_bar=False),
          NHITS(input_size=2 * horizon, h=horizon, max_steps=100, enable_progress_bar=False)]
nf = NeuralForecast(models=models, freq='1mo')
nf.fit(df=Y_train_df)
Y_hat_df = nf.predict()

In [ ]:
# Plot predictions
plot_series(Y_df.filter(pl.col("ds").le(Y_validation_df.get_column("ds").max())), Y_hat_df, engine="plotly")

## Forecast using TimeGPT

TimeGPT is a foundation model that has been trained on a large number of time series datasets. It can be used to make forecasts without training on the specific dataset.

In [ ]:
# Make a forecast
Y_hat_timegpt_df = nixtla_client.forecast(
    df=Y_train_df,
    h=horizon,
    freq="1mo",
)
Y_hat_timegpt_df.head()

In [ ]:
# Plot the forecast
plot_series(Y_df.filter(pl.col("ds").le(Y_validation_df.get_column("ds").max())), Y_hat_timegpt_df, engine="plotly")

The default model does not support long horizons. We can use the long-horizon model instead.

In [ ]:
Y_hat_timegpt_long_horizon_df = nixtla_client.forecast(
    df=Y_train_df,
    h=horizon,
    freq="1mo",
    model="timegpt-1-long-horizon"
)
Y_hat_timegpt_long_horizon_df.head()

In [ ]:
plot_series(Y_df.filter(pl.col("ds").le(Y_validation_df.get_column("ds").max())), Y_hat_timegpt_long_horizon_df, engine="plotly")

# Hands-on

- Compare TimeGPT and the neural network models
- Try different cut-offs and check if it gets better to catch the seasonality.
- Compare with the results from the first notebook
- Try what you always wanted to try using time series!

In [ ]:
# you can code here!